In [1]:
import sys
sys.path.append('../')

In [2]:
import os
import pickle
import torch
from copy import deepcopy
from pytorch_nlp_models.text_pair.siamese_rnn import SiameseGRU
from utils.datasets import LCQMCDataset
from utils.model_utils import model_train, model_eval

from torch.utils.data import DataLoader

In [3]:
DATA_PATH = '../data/'
LCQMC_PATH = os.path.join(DATA_PATH, 'LCQMC')
WORD_VECTORS_PATH = os.path.join(DATA_PATH, 'word_vectors')
BAIDUBAIKE_PKL = os.path.join(WORD_VECTORS_PATH, 'baidubaike.pkl')

MAX_SEQ_LEN = 40

MODEL_PATH = os.path.join(DATA_PATH, 'model_files')
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
    
MODEL_FILE = os.path.join(MODEL_PATH, 'siamese_gru.pkl')

In [4]:
with open(BAIDUBAIKE_PKL, 'rb') as f:
    wvs = pickle.load(f)

In [ ]:
wi = wvs['wi']
iw = wvs['iw']
dim = wvs['dim']
emb = wvs['emb']

In [ ]:
dataset = LCQMCDataset(LCQMC_PATH, MAX_SEQ_LEN, wi, charmode = True)

In [ ]:
def get_loader(dataset, mode, 
               batch_size = 32, 
               shuffle = False):
    _dataset = deepcopy(dataset)
    _dataset.to(mode)
    return DataLoader(_dataset, batch_size=batch_size, shuffle=shuffle)

In [ ]:
train_loader = get_loader(dataset, 'train',
                          shuffle= True
                         )
dev_loader = get_loader(dataset, 'dev')

In [ ]:
model = SiameseGRU(vocab_size=len(iw), emb_dim=dim)

/root/anaconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
his_train_loss, his_test_loss = model_train(model, dev_loader, dev_loader, MODEL_FILE, EPOCHS=1)

100%|██████████| 276/276 [00:02<00:00, 134.80it/s]


0/1 - train_loss: 0.5797 - test_loss: 0.3796 - test_ks: 0.6989
Test loss is reduced from inf to 0.3796. Model is saved to ../data/model_files/siamese_gru.pkl


In [ ]:
data_ks, data_auc, data_probs, data_gts = model_eval(model, dev_loader)

In [ ]:
data_ks, data_auc, data_probs, data_gts